In [ ]:
import pandas as pd 
import json
import ast
from textblob import TextBlob

In [ ]:
# Crea una lista vacía llamada data_list para almacenar los datos del archivo JSON.
data_list = []

# Especifica la ruta del archivo JSON que se va a leer.
file_path = 'australian_user_reviews.json'

# Abre el archivo en modo de lectura ('r') con codificación UTF-8.
with open(file_path, 'r', encoding='utf-8') as file:
    # Itera sobre cada línea en el archivo.
    for line in file:
        try:
            # Intenta evaluar la línea como un diccionario de Python utilizando ast.literal_eval.
            # Agrega el diccionario resultante a la lista data_list.
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            # Si ocurre un error al evaluar la línea, imprime un mensaje de error y continúa con la siguiente línea.
            print(f"Error en la línea: {line}")
            continue

# Crea un DataFrame de pandas a partir de la lista de diccionarios almacenada en data_list.
data_reviews = pd.DataFrame(data_list)

In [ ]:
# Utiliza el método 'explode' para descomponer la columna 'reviews' que contiene listas en varias filas duplicando las demás columnas.
data_reviews1 = data_reviews1.explode(['reviews'])

# Aplica el método 'pd.Series' a la columna 'reviews' para expandir las listas en nuevas columnas.
data_reviews2 = data_reviews1['reviews'].apply(pd.Series)

# Combina los DataFrames 'data_reviews1' y 'data_reviews2' a lo largo de las columnas.
data_reviews3 = pd.concat([data_reviews1, data_reviews2], axis=1)

In [ ]:

# Extrae el año de la columna 'posted' y almacénalo en una nueva columna 'year_posted'.
data_reviews3['year_posted'] = data_reviews3['posted'].str.extract('(\d{4})')

# Elimina columnas innecesarias
data_reviews3.drop(['reviews', 'user_url', 'last_edited', 'funny', 'helpful', 'posted'], axis=1, inplace=True)

# Elimina filas con valores nulos en columnas específicas
data_reviews3 = data_reviews3.dropna(subset=['year_posted', 'item_id', 'review', 'recommend'])

# Selecciona columnas específicas y renómbralas.
data_reviews3 = data_reviews3[['user_id', 'item_id', 'review', 'recommend', 'year_posted', 0]]

# Reemplaza valores 'False' y 'True' en la columna 'recommend' con 0 y 1, respectivamente.
data_reviews3['recommend'] = data_reviews3['recommend'].replace({'False': 0, 'True': 1}).astype(int)

# Elimina la última columna, que parece ser una columna residual después de expandir las listas.
del data_reviews3[data_reviews3.columns[-1]]

# Define una función para realizar análisis de sentimientos en la columna 'review'.
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment < -0.1:
        return 0
    elif sentiment > 0.1:
        return 2
    else:
        return 1

# Aplica la función 'get_sentiment' a la columna 'review' y almacena los resultados en una nueva columna 'sentiment_analysis'.
data_reviews3['sentiment_analysis'] = data_reviews3['review'].apply(get_sentiment)

# Elimina la columna original 'review' después de realizar el análisis de sentimientos.
data_reviews3.drop(['review'], axis=1, inplace=True)

# Elimina duplicados en el DataFrame.
data_reviews3 = data_reviews3.drop_duplicates()

# Elimina filas con valores nulos en todo el DataFrame.
data_reviews3 = data_reviews3.dropna()

In [ ]:
data_reviews3

In [ ]:
data_reviews3.to_csv('user_reviews.csv', index=False)